In [92]:
f = open('testforodddates.txt',"r")

In [93]:
f

<_io.TextIOWrapper name='testforodddates.txt' mode='r' encoding='cp1252'>

In [94]:
raw = f.read()

In [95]:
raw

'on Sep 30, 2015, at 9:09 AM, Jack Stock <jstock@gmail.com> wrote:\n\non Thu, Oct 1, 2015 at 1:39 PM, Wurfel, Brad <DEQ> <bigbrad@gmail.com> wrote:\n\n\non Thu, Oct 1, 2015 at 1:39 PM, Wurfel, Brad <bigbrad@gmail.com> wrote:\n\n\n\n\non Thu, Oct 1, 2015 at 1:39 PM, Brad wrote:\n\non Thu, Oct 1 at 1:39 PM, Brad wrote:\n\non Thursday at 1:39 PM, Brad wrote:\n\non Thu, Oct 1 at 1:39 PM, Brad <bigbrad@gmail.com> wrote:'

In [96]:
import urllib
import os
import re

In [97]:
def preprocess(x):
    x = x.replace('\t',' ')
    x = x.replace('\n',' ')
    x = x.replace('(',' ')
    x = x.replace(')',' ')
    x = x.replace('[',' ')
    x = x.replace(']',' ')
    x = x.replace('{',' ')
    x = x.replace('}',' ')
    x = x.replace(',',' ')
    x = x.replace('"','')
    x = x.replace("'",'')
    return(x)

In [113]:
DateTimeRegex = {'weekday month yearOrday yearORday hours:mins AMorPM':'\\b[A-Za-z]+\s+[A-Za-z]+\s+\d+\s+\d+\s+\d+\:\d+\s+[A-Z]+\\b',
                 'weekday month yearORday yearORday hours:mins:secs AMorPM':'\\b[A-Za-z]+\s+[A-Za-z]+\s+\d+\s+\d+\s+\d+\:\d+\:\d+\s+[A-Z]+\\b',
    'yearORmonthORday-yearORmonthORday-yearORmonthORday:hours:mins:secs':'\\b\d+\-\d+\-\d+\:\d+\:\d+\:\d+\\b',
             'yearORmonthORday/yearORmonthORday/yearORmonthORday:hours:mins:secs':'\\b\d+\/\d+\/\d+\:\d+\:\d+\:\d+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORday-hours.mins.secs.millisecs':'\\b\d+\-\d+\-\d+\-\d+\.\d+\.\d+\.\d+\-\d+\\b',
             'yearORmonthORday/yearORmonthORday/yearORmonthORday-hours.mins.secs.millisecs':'\\b\d+\/\d+\/\d+\-\d+\.\d+\.\d+\.\d+\-\d+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORday-hours.mins.secs':'\\b\d+\-\d+\-\d+\-\d+\.\d+\.\d+\\b',
             'yearORmonthORday/yearORmonthORday/yearORmonthORday-hours.mins.secs':'\\b\d+\/\d+\/\d+\-\d+\.\d+\.\d+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORdayThours:mins:secs.millisecs':'\\b\d+\-\d+\-\d+[A-Za-z]+\d+\:\d+\:\d+\.\w+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORdayThours:mins:secs+millisecs':'\\b\d+\-\d+\-\d+[A-Za-z]+\d+\:\d+\:\d+\+\d+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORdayThours:mins:secs*millisecs':'\\b\d+\-\d+\-\d+[A-Za-z]+\d+\:\d+\:\d+\*\d+\+\d+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORday*hours:mins:secs:millisecs':'\\b\d+\-\d+\-\d+\*\d+\:\d+\:\d+\:\d+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORday*hours:mins:secs':'\\b\d+\-\d+\-\d+\*\d+\:\d+\:\d+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORdayThours:mins:secs':'\\b\d+\-\d+\-\d+[A-Za-z]+\d+\:\d+\:\d+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORday hours:mins:secs.millisecs':'\\b\d+\-\d+\-\d+\s+\d+\:\d+\:\d+\.\w+\\b',
             'yearORmonthORday/yearORmonthORday/yearORmonthORday hours:mins:secs:millisecs':'\\b\d+\/\d+\/\d+\s+\d+\:\d+\:\d+\:\w+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORday hours:mins:secs AMorPM':'\\b\d+\-\d+\-\d+\s+\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'yearORmonthORday-yearORmonthORday-yearORmonthORday hours:mins:secs':'\\b\d+\-\d+\-\d+\s+\d+\:\d+\:\d+\\b',
             'yearORmonthORday/yearORmonthORday/yearORmonthORday hours:mins:secs AMorPM':'\\b\d+\/\d+\/\d+\s+\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'yearORmonthORday/yearORmonthORday/yearORmonthORday hours:mins:secs':'\\b\d+\/\d+\/\d+\s+\d+\:\d+\:\d+\\b',
            'yearORmonthORday/yearORmonthORday/yearORmonthORday-hours:mins':'\\b\d+\/\d+\/\d+\-\d+\.\d+\\b',
            'yearORmonthORday-yearORmonthORday-yearORmonthORday-hours:mins':'\\b\d+\-\d+\-\d+\-\d+\.\d+\\b',
            'yearORmonthORday/yearORmonthORday/yearORmonthORday:hours:mins':'\\b\d+\/\d+\/\d+\:\d+\:\d+\\b',
            'yearORmonthORday-yearORmonthORday-yearORmonthORdayThours:mins':'\\b\d+\-\d+\-\d+[A-Za-z]+\d+\:\d+\\b',
            'yearORmonthORday-yearORmonthORday-yearORmonthORday*hours:mins':'\\b\d+\-\d+\-\d+\*\d+\:\d+\\b',
            'yearORmonthORday/yearORmonthORday/yearORmonthORday hours:mins':'\\b\d+\/\d+\/\d+\s+\d+\:\d+\\b',
            'yearORmonthORday/yearORmonthORday/yearORmonthORday hours:mins AMorPM':'\\b\d+\/\d+\/\d+\s+\d+\:\d+\s+[A-Za-z]+\\b',
            'yearORmonthORday-yearORmonthORday-yearORmonthORday hours:mins':'\\b\d+\-\d+\-\d+\s+\d+\:\d+\\b',
            'yearORmonthORday-yearORmonthORday-yearORmonthORday hours:mins AMorPM':'\\b\d+\-\d+\-\d+\s+\d+\:\d+\s+[A-Za-z]\\b',
             'yearORday/month/yearORday:hours:mins:secs AMorPM':'\\b\d+\/[A-Za-z]+\/\d+\:\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'yearORday/month/yearORday:hours:mins:secs':'\\b\d+\/[A-Za-z]+\/\d+\:\d+\:\d+\:\d+\\b',
             'yearORday-month-yearORday:hours:mins:secs AMorPM':'\\b\d+\-[A-Za-z]+\-\d+\:\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'yearORday-month-yearORday:hours:mins:secs':'\\b\d+\-[A-Za-z]+\-\d+\:\d+\:\d+\:\d+\\b',
             'month/yearORday/yearORday:hours:mins:secs AMorPM':'\\b[A-Za-z]+\/\d+\/\d+\:\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'month/yearORday/yearORday:hours:mins:secs':'\\b[A-Za-z]+\/\d+\/\d+\:\d+\:\d+\:\d+\\b',
             'month-yearORday-yearORday:hours:mins:secs AMorPM':'\\b[A-Za-z]+\-\d+\-\d+\:\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'month-yearORday-yearORday:hours:mins:secs':'\\b[A-Za-z]+\-\d+\-\d+\:\d+\:\d+\:\d+\\b',
             'yearORday/month/yearORday hours:mins:secs AMorPM':'\\b\d+\/[A-Za-z]+\/\d+\s+\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'yearORday/month/yearORday hours:mins:secs':'\\b\d+\/[A-Za-z]+\/\d+\s+\d+\:\d+\:\d+\\b',
             'yearORday-month-yearORday hours:mins:secs AMorPM':'\\b\d+\-[A-Za-z]+\-\d+\s+\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'yearORday-month-yearORday hours:mins:secs':'\\b\d+\-[A-Za-z]+\-\d+\s+\d+\:\d+\:\d+\\b',
             'month/yearORday/yearORday hours:mins:secs AMorPM':'\\b[A-Za-z]+\/\d+\/\d+\s+\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'month/yearORday/yearORday hours:mins:secs':'\\b[A-Za-z]+\/\d+\/\d+\s+\d+\:\d+\:\d+\\b',
             'month-yearORday-yearORday hours:mins:secs AMorPM':'\\b[A-Za-z]+\-\d+\-\d+\s+\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'month-yearORday-yearORday hours:mins:secs':'\\b[A-Za-z]+\-\d+\-\d+\s+\d+\:\d+\:\d+\\b',
             'yearORday month yearORday hours:mins:secs.millisecs':'\\b\d+\s+[A-Za-z]+\s+\d+\s+\d+\:\d+\:\d+\.\d+\\b',
             'month dayORyear hours:mins:secs +millisecs dayORyear':'\\b[A-Za-z]+\s+\d+\s+\d+\:\d+\:\d+\s+\+\d+\s+\d+\\b',
             'month dayORyear hours:mins:secs dayORyear':'\\b[A-Za-z]+\s+\d+\s+\d+\:\d+\:\d+\s+\d+\\b',
             'month dayORyear dayORyear hours:mins:secs AMorPM':'\\b[A-Za-z]+\s+\d+\s+\d+\s+\d+\:\d+\:\d+\s+[A-Z]+\\b',
             'month dayORyear dayORyear hours:mins:secs':'\\b[A-Za-z]+\s+\d+\s+\d+\s+\d+\:\d+\:\d+\\b',
             'month dayORyear hours:mins:secs +millisecs':'\\b[A-Za-z]+\s+\d+\s+\d+\:\d+\:\d+\s+\+\d+\\b',
             'dayORyear month dayORyear hours:mins:secs':'\\b\d+\s+[A-Za-z]+\s+\d+\s+\d+\:\d+\:\d+\\b',
             'month dayORyear hours:mins:secs':'\\b[A-Za-z]+\s+\d+\s+\d+\:\d+\:\d+\\b',
             'yearORmonthORday/yearORmonthORday/yearORmonthORday':'\\b\d+\/\d+\/\d+\*\d+\:\d+\:\d+\\b',
'yearORday/month/yearORday:hours:mins AMorPM':'\\b\d+\/[A-Za-z]+\d+\:\d+\:\d+\s+[A-Za-z]+\\b',
'yearORday/month/yearORday:hours:mins':'\\b\d+\/[A-Za-z]+\/\d+\:\d+\:\d+\\b',
'yearORday-month-yearORday:hours:mins AMorPM':'\\b\d+\-[A-Za-z]+\-\d+\:\d+\:\d+\s+[A-Za-z]+\\b',
'yearORday-month-yearORday:hours:mins':'\\b\d+\-[A-Za-z]+\-\d+\:\d+\:\d+\\b',
'month-yearORday-yearORday:hours:mins AMorPM':'\\b[A-Za-z]+\-\d+\-\d+\:\d+\:\d+\s+[A-Za-z]+\\b',
'month-yearORday-yearORday:hours:mins':'\\b[A-Za-z]+\-\d+\-\d+\:\d+\:\d+\\b',
'month/yearORday/yearORday:hours:mins AMorPM':'\\b[A-Za-z]+\/\d+\/\d+\:\d+\:\d+\s+[A-Za-z]+\\b',
'month/yearORday/yearORday:hours:mins':'\\b[A-Za-z]+\/\d+\/\d+\:\d+\:\d+\\b',
'yearORday/month/yearORday hours:mins AMorPM':'\\b\d+\/[A-Za-z]+\/\d+\s+\d+\:\d+\s+[A-Za-z]+\\b',
'yearORday/month/yearORday hours:mins':'\\b\d+\/[A-Za-z]+\/\d+\s+\d+\:\d+\\b',
'yearORday-month-yearORday hours:mins AMorPM':'\\b\d+\-[A-Za-z]+\-\d+\s+\d+\:\d+\s+[A-Za-z]+\\b',
'yearORday-month-yearORday hours:mins':'\\b\d+\-[A-Za-z]+\-\d+\s+\d+\:\d+\\b',
'month/yearORday/yearORday hours:mins AMorPM':'\\b[A-Za-z]\/\d+\/\d+\s+\d+\:\d+\s+[A-Za-z]+\\b',
'month/yearORday/yearORday hours:mins':'\\b[A-Za-z]\/\d+\/\d+\s+\d+\:\d+\\b',
'month-yearORday-yearORday hours:mins AMorPM':'\\b[A-Za-z]+\-\d+\-\d+\s+\d+\:\d+\s+[A-Za-z]+\\b',
'month-yearORday-yearORday hours:mins':'\\b[A-Za-z]+\-\d+\-\d+\s+\d+\:\d+\\b',
'year month day hours:mins':'\\b\d+\s+[A-Za-z]+\s+\d+\s+\d+\:\d+\\b',
'month day hours:mins year':'\\b[A-Za-z]+\s+\d+\s+\d+\:\d+\s+\d+\\b',
'month day hours:mins':'\\b[A-Za-z]+\s+\d+\s+\d+\:\d+\\b',
'month day year hours:mins AMorPM':'\\b[A-Za-z]+\s+\d+\s+\d+\s+\d+\:\d+\s+[A-Za-z]+\\b',
'month day year hours:mins':'\\b[A-Za-z]+\s+\d+\s+\d+\s+\d+\:\d+\\b',
'on weekday, month day, year at hours:mins AMorPM':'\\b[a-z]+\s+[A-Za-z]+\s+[A-Za-z]+\s+\d+\s+\d+\s+[a-z]+\s+\d+\:\d+\s+[A-Za-z]+\\b',
'on month day, year, at hours:mins AMorPM':'\\b[a-z]+\s+[A-Za-z]+\s+\d+\s+\d+\s+[a-z]+\s+\d+\:\d+\s+[A-Za-z]+\\b'}

In [114]:
reg = '|'.join(DateTimeRegex.values())

In [115]:
def DateTimeExtractor(x):
    x = preprocess(x)
    DT = re.findall(reg,x)
    return(DT)

In [116]:
result = DateTimeExtractor(raw)

In [117]:
print(result)

['on Sep 30  2015  at 9:09 AM', 'on Thu  Oct 1  2015 at 1:39 PM', 'on Thu  Oct 1  2015 at 1:39 PM', 'on Thu  Oct 1  2015 at 1:39 PM']
